# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

In [ ]:
# !pip install tensorly

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/) - само аналитическое разложение вам реализовывать НЕ НУЖНО

In [ ]:
import random
import time
import torch
import pandas as pd
import numpy as np

import tensorly as tl
from tensorly.decomposition import non_negative_tucker, non_negative_tucker_hals, tucker
from tensorly.metrics.regression import RMSE

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

import math
from torch.optim.optimizer import Optimizer
import torch.nn.functional as F

np.random.seed(0)
torch.manual_seed(0)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [ ]:
# # Функция, восстанавливающая тензор по ядру и матрицам
# def repair_tensor(G_, U):
#     # data - восстановленный тензор из матриц и ядра
#     # U - список матриц
#     # G_ - ядро разложения
#     a1 = tl.tenalg.mode_dot(tensor=tl.tensor(G_.detach().numpy()), matrix_or_vector=tl.tensor(U[0].detach().numpy()), mode=0, transpose=False)
#     a2 = tl.tenalg.mode_dot(tensor=a1, matrix_or_vector=tl.tensor(U[1].detach().numpy()), mode=1, transpose=False)
#     a3 = tl.tenalg.mode_dot(tensor=a2, matrix_or_vector=tl.tensor(U[2].detach().numpy()), mode=2, transpose=False)
#     return torch.tensor(a3, dtype=torch.double, requires_grad=True, device=device)

In [ ]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=10):
    # data - тензор с заданной размерностью
    # U - список матриц
    U = [torch.tensor(np.random.randint(10, size=(size[i], r)), dtype=torch.double, requires_grad=True, device=device) for i in range(len(size))]
    # G - ядро разложения
    G = torch.tensor(np.random.randint(10, size=(r, r, r)), dtype=torch.double, requires_grad=True, device=device)
    data = rebuild_tensor(G, U)

    return data, U, G

In [ ]:
def rebuild_tensor(G_, U):
    result = G_
    print(f"Core tensor requires_grad: {G_.requires_grad}")
    for i, u in enumerate(U):
        result = torch.tensordot(result, u, dims=([0], [1]))
        print(f"Factor {i} requires_grad: {u.requires_grad}")
    print(f"result tensor requires_grad: {result.requires_grad}")
    return result

In [ ]:
a = torch.tensor(np.random.normal(0, 1e-2, size=size), dtype=torch.double, device=device)
a.shape

torch.Size([10, 20, 30])

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [ ]:
# size = (100, 200, 300)
# r = 10

size = (10, 20, 30)
r = 5

data, U, G = get_tensor(size, r)
data.shape, [u.shape for u in U], G.shape

Core tensor requires_grad: True
Factor 0 requires_grad: True
Factor 1 requires_grad: True
Factor 2 requires_grad: True
result tensor requires_grad: True


(torch.Size([10, 20, 30]),
 [torch.Size([10, 5]), torch.Size([20, 5]), torch.Size([30, 5])],
 torch.Size([5, 5, 5]))

In [ ]:
noise = torch.tensor(np.random.normal(0, 1e-2, size=size), dtype=torch.double, device=device)
data_w_noise = data + noise
print(f"data_w_noise requires_grad: {data_w_noise.requires_grad}")

data_w_noise requires_grad: True


Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ:

## 2 Реализуйте метод для восстановления тензора по разложению

In [ ]:
# Функция, восстанавливающая тензор по ядру и матрицам (для результата из библы tensorly)
def repair_tensor_for_ndarray(G_, U):
    # data - восстановленный тензор из матриц и ядра
    # U - список матриц
    # G_ - ядро разложения
    a1 = tl.tenalg.mode_dot(tensor=tl.tensor(G_), matrix_or_vector=tl.tensor(U[0]), mode=0, transpose=False)
    a2 = tl.tenalg.mode_dot(tensor=a1, matrix_or_vector=tl.tensor(U[1]), mode=1, transpose=False)
    a3 = tl.tenalg.mode_dot(tensor=a2, matrix_or_vector=tl.tensor(U[2]), mode=2, transpose=False)
    return torch.tensor(a3, dtype=torch.double)

## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [ ]:
# использую tucker from tensorly
data_ndarray = data_w_noise.detach().numpy()
core, factors = tucker(tl.tensor(data_ndarray), [r, r, r])
print(core.shape, [u.shape for u in factors])

(5, 5, 5) [(10, 5), (20, 5), (30, 5)]


Не забудьте померить ошибку разложения по метрике MSE

In [ ]:
def MSE(tensor1, tensor2):
    delta = tensor1-tensor2
    delta *= delta
    mse = delta.sum() / delta.numel()
    return mse.item()

In [ ]:
repaired_data = repair_tensor_for_ndarray(core, factors)

In [ ]:

MSE(repaired_data, data_w_noise)

9.538384390111432e-05

## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [ ]:
# class Opt(Optimizer):

#     def __init__(self, params, lr=1e-3, ...):
#         super().__init__(params, defaults)

#     def step(self):
#         return loss

### 4.2 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [ ]:
class TuckerOptimizer(Optimizer):
    def __init__(self, params, lr=1e-3):
        defaults = {'lr': lr}
        super().__init__(params, defaults)

    # по мотивам доков sgd optimizer
    def step(self, closure=None):
        """Perform a single optimization step.

        Args:
            closure (Callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()


        for group in self.param_groups:
            lr = group['lr']
            for param in group['params']:
                print(param.grad)
                if param.grad is not None:
                    param.data -= lr * param.grad

        return loss


In [ ]:
def rebuild_tensor(G_, U):
    result = G_
    print(f"Core tensor requires_grad: {G_.requires_grad}")
    for i, u in enumerate(U):
        result = torch.tensordot(result, u, dims=([0], [1]))
        print(f"Factor {i} requires_grad: {u.requires_grad}")
    print(f"result tensor requires_grad: {result.requires_grad}")
    return result

In [ ]:
tensor = data_w_noise  # Исходный тензор
# Ядро Таккера
core_tensor = torch.tensor(np.random.randint(10, size=(r, r, r)), dtype=torch.double, requires_grad=True, device=device)

# Факторы Таккера (те самые случайные матрицы требующие градиентной оптимизации)
factors = [torch.tensor(np.random.randint(10, size=(size[i], r)),
        dtype=torch.double, requires_grad=True, device=device) for i in range(len(size))]


# Инициализация оптимизатора
optimizer = TuckerOptimizer([core_tensor] + factors, lr=1e-3)


for epoch in range(50):

    print(f"Core tensor requires_grad: {core_tensor.requires_grad}")
    for i, factor in enumerate(factors):
        print(f"Factor {i} requires_grad: {factor.requires_grad}")

    optimizer.zero_grad()

    # Перестроение тензора
    approx_tensor = rebuild_tensor(core_tensor, factors)

    # Вычисление квадратичного лосса
    loss = F.mse_loss(approx_tensor, tensor)

    # Вычисление градиентов
    loss.backward()

    print(loss)
    # Шаг оптимизации
    optimizer.step()
    print(loss)


    if epoch % 10 == 0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

Core tensor requires_grad: True
Factor 0 requires_grad: True
Factor 1 requires_grad: True
Factor 2 requires_grad: True
Core tensor requires_grad: True
Factor 0 requires_grad: True
Factor 1 requires_grad: True
Factor 2 requires_grad: True
result tensor requires_grad: True
tensor(1.1986e+09, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor([[[  -84291.0126,  -999485.8621,  -576736.4254, -1003162.8396,
           -323033.3616],
         [  670754.8501,  -241213.0115,   187587.1444,  -245517.7817,
            476300.4326],
         [  209319.6851,  -611716.7582,  -229383.7936,  -674428.1402,
            -27256.3777],
         [    2726.5110,  -724843.5425,  -438011.0033,  -690819.7143,
           -223279.9614],
         [ -132704.6594, -1389867.5332,  -852601.4254, -1367610.2119,
           -459350.4063]],

        [[ 1349603.9791,   938714.7154,  1238817.2858,   973544.1509,
           1389741.0332],
         [ 2113913.3086,  1737131.1509,  2050416.0155,  1765323.1649,
           2

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:

# Оптимизация
# for epoch in range(50):
#     def closure():
#         optimizer.zero_grad()
#         # Перестроение тензора на основе текущего ядра и факторов
#         approx_tensor = rebuild_tensor(core_tensor, factors)
#         # Вычисление квадратичного лосса
#         loss = F.mse_loss(approx_tensor, tensor)
#         loss.backward()
#         return loss

#     loss = optimizer.step(closure)
#     if epoch%10 == 0:
#         print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

## 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.